## Setup

In [1]:
import re
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
from alphalens.utils import get_clean_factor_and_forward_returns
from alphalens.performance import *
from alphalens.plotting import *
from alphalens.tears import *

In [2]:
warnings.filterwarnings('ignore')
%matplotlib inline
plt.style.use('fivethirtyeight')

### Version conflict

At the time of writing, `zipline` required `pandas<=0.22` so you may need to run `pip install -U zipline` to temporarily downgrade `pandas` or set up a separate environment.

In [3]:
# !pip install -U zipline

## Zipline AlphaFactor Test

We are first going to illustrate the zipline alpha factor research workflow in an offline environment. In particular, we will develop and test a simple mean-reversion factor that measures how much recent performance has deviated from the historical average. 

Short-term reversal is a common strategy that takes advantage of the weakly predictive pattern that stock price increases are likely to mean-revert back down over horizons from less than a minute to one month.

To this end, the factor computes the z-score for the last monthly return relative to the rolling monthly returns over the last year. At this point, we will not place any orders to simply illustrate the implementation of a CustomFactor and record the results during the simulation.

After some basic settings, `MeanReversion` subclasses `CustomFactor` and defines a `compute()` method. It creates default inputs of monthly returns over an also default year-long window so that the monthly_return variable will have 252 rows and one column for each security in the Quandl dataset on a given day.

The `compute_factors()` method creates a `MeanReversion` factor instance and creates long, short, and ranking pipeline columns. The former two contain Boolean values that could be used to place orders, and the latter reflects that overall ranking to evaluate the overall factor performance. Furthermore, it uses the built-in `AverageDollarVolume` factor to limit the computation to more liquid stocks

The result would allow us to place long and short orders. We will see in the next chapter how to build a portfolio by choosing a rebalancing period and adjusting portfolio holdings as new signals arrive.

- The `initialize()` method registers the compute_factors() pipeline, and the before_trading_start() method ensures the pipeline runs on a daily basis. 
- The `record()` function adds the pipeline's ranking column as well as the current asset prices to the performance DataFrame returned by the `run_algorithm()` function

We will use the factor and pricing data stored in the performance DataFrame to evaluate the factor performance for various holding periods in the next section, but first, we'll take a look at how to create more complex signals by combining several alpha factors from a diverse set of data sources on the Quantopian platform.

Run using jupyter notebook extension

In [5]:
%load_ext zipline

In [6]:
%%zipline --start 2015-1-1 --end 2018-1-1 --output single_factor.pickle

from zipline.api import (
    attach_pipeline,
    date_rules,
    time_rules,
    order_target_percent,
    pipeline_output,
    record,
    schedule_function,
    get_open_orders,
    calendars
)
from zipline.finance import commission, slippage
from zipline.pipeline import Pipeline, CustomFactor
from zipline.pipeline.factors import Returns, AverageDollarVolume
import numpy as np
import pandas as pd

MONTH = 21
YEAR = 12 * MONTH
N_LONGS = N_SHORTS = 25
VOL_SCREEN = 1000


class MeanReversion(CustomFactor):
    """Compute ratio of latest monthly return to 12m average,
       normalized by std dev of monthly returns"""
    inputs = [Returns(window_length=MONTH)]
    window_length = YEAR

    def compute(self, today, assets, out, monthly_returns):
        df = pd.DataFrame(monthly_returns)
        out[:] = df.iloc[-1].sub(df.mean()).div(df.std())


def compute_factors():
    """Create factor pipeline incl. mean reversion,
        filtered by 30d Dollar Volume; capture factor ranks"""
    mean_reversion = MeanReversion()
    dollar_volume = AverageDollarVolume(window_length=30)
    return Pipeline(columns={'longs': mean_reversion.bottom(N_LONGS),
                             'shorts': mean_reversion.top(N_SHORTS),
                             'ranking': mean_reversion.rank(ascending=False)},
                    screen=dollar_volume.top(VOL_SCREEN))


def exec_trades(data, assets, target_percent):
    """Place orders for assets using target portfolio percentage"""
    for asset in assets:
        if data.can_trade(asset) and not get_open_orders(asset):
            order_target_percent(asset, target_percent)


def rebalance(context, data):
    """Compute long, short and obsolete holdings; place trade orders"""
    factor_data = context.factor_data
    record(factor_data=factor_data.ranking)

    assets = factor_data.index
    record(prices=data.current(assets, 'price'))

    longs = assets[factor_data.longs]
    shorts = assets[factor_data.shorts]
    divest = set(context.portfolio.positions.keys()) - set(longs.union(shorts))

    exec_trades(data, assets=divest, target_percent=0)
    exec_trades(data, assets=longs, target_percent=1 / N_LONGS)
    exec_trades(data, assets=shorts, target_percent=-1 / N_SHORTS)


def initialize(context):
    """Setup: register pipeline, schedule rebalancing,
        and set trading params"""
    attach_pipeline(compute_factors(), 'factor_pipeline')
    schedule_function(rebalance,
                      date_rules.week_start(),
                      time_rules.market_open(),
                      calendar=calendars.US_EQUITIES)
    context.set_commission(commission.PerShare(cost=.01, min_trade_cost=0))
    context.set_slippage(slippage.VolumeShareSlippage())


def before_trading_start(context, data):
    """Run factor pipeline"""
    context.factor_data = pipeline_output('factor_pipeline')

,algo_volatility,algorithm_period_return,alpha,benchmark_period_return,benchmark_volatility,beta,capital_used,ending_cash,ending_exposure,ending_value,...,short_exposure,short_value,shorts_count,sortino,starting_cash,starting_exposure,starting_value,trading_days,transactions,treasury_period_return
2015-01-02 21:00:00+00:00,NaN,0.000000,NaN,-0.000535,NaN,NaN,0.000000e+00,1.000000e+07,0.000,0.000,...,0.000,0.000,0,NaN,1.000000e+07,0.000,0.000,1,[],0.0
2015-01-05 21:00:00+00:00,0.004462,-0.000398,0.003059,-0.018585,0.196712,0.022684,-3.471997e+06,6.528003e+06,3468022.250,3468022.250,...,-1584674.470,-1584674.470,4,-11.224972,1.000000e+07,0.000,0.000,2,"[{'dt': 2015-01-05 21:00:00+00:00, 'amount': 5...",0.0
2015-01-06 21:00:00+00:00,0.087450,-0.010128,-0.785839,-0.027829,0.139101,0.027715,2.579878e+05,6.785990e+06,3112732.065,3112732.065,...,-1616701.625,-1616701.625,4,-9.531491,6.528003e+06,3468022.250,3468022.250,3,"[{'dt': 2015-01-06 21:00:00+00:00, 'amount': 9...",0.0
2015-01-07 21:00:00+00:00,0.080463,-0.008845,-0.401660,-0.015715,0.206972,0.158171,0.000000e+00,6.785990e+06,3125561.843,3125561.843,...,-1629422.430,-1629422.430,4,-7.198541,6.785990e+06,3112732.065,3112732.065,4,[],0.0
2015-01-08 21:00:00+00:00,0.092816,-0.002474,-0.150124,0.001751,0.236040,0.260216,0.000000e+00,6.785990e+06,3189267.130,3189267.130,...,-1644535.790,-1644535.790,4,-1.754848,6.785990e+06,3125561.843,3125561.843,5,[],0.0
2015-01-09 21:00:00+00:00,0.085714,-0.006238,-0.194003,-0.006276,0.218111,0.268366,0.000000e+00,6.785990e+06,3151628.598,3151628.598,...,-1670536.830,-1670536.830,4,-3.834416,6.785990e+06,3189267.130,3189267.130,6,[],0.0
2015-01-12 21:00:00+00:00,0.078270,-0.007585,-0.143842,-0.014061,0.203321,0.259298,0.000000e+00,6.785990e+06,3138163.975,3138163.975,...,-1659398.310,-1659398.310,4,-4.292170,6.785990e+06,3151628.598,3151628.598,7,[],0.0
2015-01-13 21:00:00+00:00,0.074637,-0.005492,-0.037813,-0.016834,0.188301,0.256687,1.675249e+06,8.461239e+06,1483843.830,1483843.830,...,-1600072.490,-1600072.490,4,-2.891468,6.785990e+06,3138163.975,3138163.975,8,"[{'dt': 2015-01-13 21:00:00+00:00, 'amount': 1...",0.0
2015-01-14 21:00:00+00:00,0.070191,-0.007529,-0.046792,-0.022769,0.177393,0.257921,4.236868e+04,8.503608e+06,1421106.150,1421106.150,...,-1571634.240,-1571634.240,4,-3.685843,8.461239e+06,1483843.830,1483843.830,9,"[{'dt': 2015-01-14 21:00:00+00:00, 'amount': -...",0.0
2015-01-15 21:00:00+00:00,0.066363,-0.007372,0.008937,-0.031721,0.170557,0.242310,0.000000e+00,8.503608e+06,1422668.199,1422668.199,...,-1546157.361,-1546157.361,4,-3.423078,8.503608e+06,1421106.150,1421106.150,10,[],0.0
